## Полезные библиотеки и ссылки:

https://pypi.org/project/cigar/

difflib

https://pypi.org/project/Edit_Distance/

combinations

https://pypi.org/project/fuzzysearch/

http://ginstrom.com/scribbles/2007/12/01/fuzzy-substring-matching-with-levenshtein-distance-in-python/

https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm

https://pypi.org/project/regex/

In [1]:
from fuzzysearch import find_near_matches
from itertools import combinations, tee
from itertools import combinations
from copy import copy, deepcopy

In [2]:
def binary_generator(array):
    i = -1
    yield i
    for x in array:
        i += 1
        if x == False:
            yield i 

In [3]:
def D_seq(string, i):
    return string[:i] + string[i+1:]
def I_seq(string, i, element):
    return string[:i] + element + string[i:]
def X_seq(string, i, element):
    return string[:i] + element + string[i+1:]

In [4]:
import edit_distance
def cigar(s1, s2):
    cigar_string = ""
    sm = edit_distance.SequenceMatcher(s1, s2).get_opcodes()
    #print(cigar_string)
    for opcode in sm:
        if opcode[0] == 'equal': 
            if len(cigar_string) > 0:
                if cigar_string[-1] == 'M':
                    cigar_string = change_string(cigar_string)        
                else:
                    cigar_string += '1M'
            else:
                cigar_string += '1M'
        elif opcode[0] == 'replace': 
            if len(cigar_string) > 0:
                if cigar_string[-1] == 'X':
                    cigar_string = change_string(cigar_string)
                else:
                    cigar_string += '1X'
            else:
                cigar_string += '1X'
        elif opcode[0] == 'insert': 
            if len(cigar_string) > 0:
                if cigar_string[-1] == 'I':
                    cigar_string = change_string(cigar_string)
                else:
                    cigar_string += '1I'
            else:
                cigar_string += '1I'
        elif opcode[0] == 'delete': 
            if len(cigar_string) > 0:
                if cigar_string[-1] == 'D':
                    cigar_string = change_string(cigar_string)
                else:
                    cigar_string += '1D'
            else:
                cigar_string += '1D'
        #print(cigar_string)
    return cigar_string

In [5]:
def change_string(s):
    return s[:-cigar_add(s)-1] + str(int(s[-cigar_add(s)-1:-1])+1) + s[-1]

In [6]:
def cigar_add(cigar_string):
    if len(cigar_string) >= 3:
        if cigar_string[-3] != 'M' and cigar_string[-3] != 'X' and cigar_string[-3] != 'I' and \
        cigar_string[-3] != 'D':
            return cigar_add(cigar_string[:-1]) + 1
        else:
            return 1
    else:
        return 1

In [7]:
def cigar_to_binary(cigar):
    binary = []
    i = 0; k = 0
    while k < len(cigar):
        while cigar[k] != 'M' and cigar[k] != 'X' and cigar[k] != 'I' and cigar[k] != 'D':
            k += 1
        if cigar[k] == 'M':
            for j in range(int(cigar[i:k])):
                binary.append(True)
        else:
            for j in range(int(cigar[i:k])):
                binary.append(False)
        k += 1
        i = k
    return binary

In [8]:
def intersection(cigars):
    l = len(cigar_to_binary(cigars[0]))
    res = [True for i in range(l)]
    for cigar in cigars:
        binary = cigar_to_binary(cigar)
        res = [res[i] and binary[i] for i in range(l)]
    return res

In [30]:
class Sequence_t:
    def __init__(self, file):
        f = open(file, 'r')
        numbers = [int(number) for number in f.readline().rstrip().split(" ")]
        self.sequence = f.readline()
        self.n = numbers[0]
        self.l = numbers[1]
        self.d = numbers[2]
        self.L = len(self.sequence)
        f.close()
    def main_sample(self):
        results = []
        for i in range(self.L - self.n * self.l):
            result = find_near_matches(self.sequence[i:i+self.l], \
                                       self.sequence[i:self.L], max_l_dist=self.d*2)
            if(len(result) >= self.n):
                results.append((result, [i + x[0] for x in result], [i + x[1] for x in result]))
        print(results)
        return results
    def search_subsequence(self, subsequence, generator):
        DNA = 'ATGC'
        try:
            i = next(generator)
        except StopIteration:
            return [], []
        generators = [1, 2, 3, 4]
        for j in range(4):
            generator, generators[j] = tee(generator)
        for x, generator_ in zip(DNA, generators):
            subsequence_ = X_seq(subsequence, i, x)
            result = find_near_matches(subsequence_, self.sequence, max_l_dist=self.d)
            if len(result) >= self.n:
                return result, subsequence_
            else:
                result, subsequence_ = self.search_subsequence(subsequence_, generator_)
                if result != []:
                    return result, subsequence_
        return result, subsequence_
    def search_E(self, starts, ends):
        subsequences = []
        if starts[0] <= self.d:
            E = self.sequence[starts[1] - self.d:ends[1] + self.d]
            subsequences.append('A'*d + self.sequence[0:ends[0] + self.d])
            for i in range(2, len(starts)):
                subsequences.append(self.sequence[starts[i] - self.d:ends[i] + self.d])
        else:
            E = self.sequence[starts[0] - self.d:ends[0] + self.d]
            subsequences = [self.sequence[starts[i] - self.d : ends[i] + self.d] for i in range(1, len(starts))]
        intersections_ = intersection([cigar(subsequence, E) for subsequence in subsequences])
        for i in range(2*self.d+1):
            print(i)
            if i == 0:
                g = binary_generator(intersections_)
                next(g)
                result, subsequence_ = self.search_subsequence(E, g)
                if(result != []):
                    return result, subsequence_
            else:
                full = binary_generator(intersections_)
                next(full)
                for comb in combinations([j for j in full], i):
                    E_ = E
                    intersections__ = deepcopy(intersections_)
                    num = 0
                    for k in comb:
                        E_ = D_seq(E_, k - num)
                        del intersections__[k]
                        num += 1
                    g = binary_generator(intersections__)
                    next(g)
                    result, subsequence_ = self.search_subsequence(E_, g)
                    if(result != []):
                        return result, subsequence_
        return result, subsequence_
    def output(self, file):
        for result, starts, ends in self.main_sample():
            result, sequence = self.search_E(starts, ends)
            if result != []:
                break
        output = open(file, 'w')
        output.write(sequence + '\n')
        for x in result:
            output.write(str(x[0]+1) + ' ' + cigar(self.sequence[x[0]:x[1]], sequence) + '\n')
        output.close()
        return sequence

In [31]:
sequence3 = Sequence_t('3.txt')
sequence3.output('result_3.txt')

[([Match(start=0, end=10, dist=0), Match(start=20, end=29, dist=1), Match(start=42, end=51, dist=1), Match(start=65, end=73, dist=2)], [3, 23, 45, 68], [13, 32, 54, 76]), ([Match(start=0, end=10, dist=0), Match(start=20, end=29, dist=1), Match(start=42, end=51, dist=1), Match(start=65, end=73, dist=2)], [4, 24, 46, 69], [14, 33, 55, 77])]
0
1
2


'ATGCAGCATT'

In [22]:
sequence2 = Sequence_t('2.txt')
sequence2.output('result_2.txt')

[([Match(start=0, end=30, dist=0), Match(start=78, end=108, dist=2), Match(start=121, end=150, dist=2)], [7, 85, 128], [37, 115, 157]), ([Match(start=0, end=30, dist=0), Match(start=78, end=108, dist=1), Match(start=119, end=150, dist=2)], [8, 86, 127], [38, 116, 158]), ([Match(start=0, end=30, dist=0), Match(start=79, end=107, dist=2), Match(start=120, end=150, dist=1)], [9, 88, 129], [39, 116, 159]), ([Match(start=0, end=30, dist=0), Match(start=79, end=108, dist=2), Match(start=121, end=149, dist=2)], [10, 89, 131], [40, 118, 159]), ([Match(start=0, end=30, dist=0), Match(start=78, end=108, dist=1), Match(start=120, end=150, dist=1)], [11, 89, 131], [41, 119, 161]), ([Match(start=0, end=30, dist=0), Match(start=78, end=109, dist=2), Match(start=120, end=149, dist=2)], [12, 90, 132], [42, 121, 161])]
0
1


'CAGGCTAGACCATCCTCTTGCTTGCCCCTAC'

In [17]:
sequence5 = Sequence_t('5.txt')
sequence5.output('result_5.txt')

[([Match(start=0, end=10, dist=0), Match(start=48, end=60, dist=2), Match(start=85, end=96, dist=4), Match(start=117, end=124, dist=4)], [5, 53, 90, 122], [15, 65, 101, 129]), ([Match(start=0, end=10, dist=0), Match(start=34, end=44, dist=4), Match(start=55, end=64, dist=5), Match(start=65, end=76, dist=6), Match(start=98, end=106, dist=5)], [37, 71, 92, 102, 135], [47, 81, 101, 113, 143]), ([Match(start=0, end=10, dist=0), Match(start=34, end=44, dist=4), Match(start=54, end=65, dist=6), Match(start=66, end=74, dist=5), Match(start=97, end=105, dist=5)], [39, 73, 93, 105, 136], [49, 83, 104, 113, 144]), ([Match(start=0, end=10, dist=0), Match(start=34, end=44, dist=5), Match(start=54, end=60, dist=6), Match(start=65, end=77, dist=5), Match(start=96, end=105, dist=5)], [40, 74, 94, 105, 136], [50, 84, 100, 117, 145]), ([Match(start=0, end=10, dist=0), Match(start=35, end=42, dist=5), Match(start=65, end=77, dist=5), Match(start=96, end=108, dist=6)], [41, 76, 106, 137], [51, 83, 118, 1

IndexError: list index out of range